# Poisson problem

Let us solve the Poisson problem of finding $u$ satisfying 

$$
\begin{aligned}
-\Delta u + u& = f && \text { in  } (0,10)^2,
\\
\frac{\partial u }{\partial n } & = 0 
&& \text{ on the boundary parts}.
\end{aligned}
$$

## Quick steps to solution:

#### 1. Import NGSolve and Netgen Python modules:

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw

#### 2. Main discretization parameters and mesh

In [ ]:
order = 4
maxh = 0.5

from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle((0,0), (10,10), bcs=['bottom','right','top','left'])

mesh = Mesh(geo.GenerateMesh(maxh=maxh))
mesh.nv, mesh.ne

#### 3. Reference solution and corresponding right hand side


In [ ]:
ref_sol = exp(-2*((x-4)**2+(y-6)**2))
Draw(ref_sol, mesh, "reference solution")

refdx = ref_sol.Diff(x)
refdy = ref_sol.Diff(y)
rhs = -refdx.Diff(x)-refdy.Diff(y) + ref_sol

ref_sol_grad = CoefficientFunction( (refdx,refdy) )

#### 4. Declare a finite element space and gridfunction

In [ ]:
fes = H1(mesh, order=order)# fe space
sol = GridFunction(fes)  # solution 
print(fes.ndof) # number of degrees of freedom

#### 5. Define and assemble linear and bilinear forms

In [ ]:
u = fes.TrialFunction()  # symbolic object
v = fes.TestFunction()   # symbolic object

a = BilinearForm(fes, symmetric=True)
a += (grad(u)*grad(v) + u*v)*dx
a.Assemble()

f = LinearForm(fes)
f += rhs*v*dx
f.Assemble()

#### 6. Solve system and draw solution

In [ ]:
from time import time

t1 = time()
sol.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec
print("time = ", time()-t1)
Draw(sol, mesh, "sol")

#### 7. Compute error

In [ ]:
Draw(sqrt((ref_sol-sol)**2 + InnerProduct(ref_sol_grad-Grad(sol),ref_sol_grad-Grad(sol))), mesh, 'error')
H1err = sqrt( Integrate((ref_sol-sol)**2 + InnerProduct(ref_sol_grad-Grad(sol),ref_sol_grad-Grad(sol)), mesh, VOL, 2*order) )
help(Integrate)
print(H1err)

#### 8. Draw error plot

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

plt.plot(fes.ndof, H1err, 'x')

plt.yscale('log')
plt.xscale('log')
plt.xlabel("ndof")
plt.ylabel("error")
plt.show()